In [94]:
import numpy as np
import pandas as pd
from pandas_datareader import data as web
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
import datetime as dt
import mplfinance as mpf
import yfinance as yf

In [37]:
pip install mplfinance

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'd:\py\venv\scripts\python.exe -m pip install --upgrade pip' command.


  Created wheel for mpf: filename=mpf-0.55.0-cp34-none-any.whl size=1451518 sha256=b8e7af438d6cd3f06f81f5f8d0d3f31674ed1bacaaf05d819f0d2ddb44f43636
  Stored in directory: c:\users\runji\appdata\local\pip\cache\wheels\2c\94\8c\494aa8275ac668aa50d41846d2e7e63895135be02413126053
  Created wheel for ruamel.yaml: filename=ruamel.yaml-0.15.100-cp38-cp38-win_amd64.whl size=232499 sha256=063a82291f2b863b88f2775b5c1b1c813d5d5bfc6219b78b88df64b87111980f
  Stored in directory: c:\users\runji\appdata\local\pip\cache\wheels\83\bb\c8\84a46029883fd202fda2e3d0a579eb30641fecb325b9907fb0
Successfully built mpf ruamel.yaml


You should consider upgrading via the 'd:\py\venv\scripts\python.exe -m pip install --upgrade pip' command.


# **Function to save stock data to CSV**


In [45]:
def save_to_csv_from_yahoo(ticker, sdate, edate):
  #start = dt.datetime(sdate)
  #end = dt.datetime(edate)

  #tickerData = yf.Ticker(ticker)
  
  # get historical market data
  #df = tickerData.history(period="max")
  yfdf = yf.download(ticker, start=sdate, end=edate)
  #yfdf = yf.download('gbtc', start="2021-01-01", end="2021-01-01")
  #print(yfdf)
  mydf = pd.DataFrame(yfdf)
  

  #yfdf.actions.to_csv('D:\\py\\pycode\\banas-tut01\\' + ticker + '.csv')
  #csvresult = mydf.to_csv('D:\\py\\pycode\\banas-tut01\\' + ticker + '.csv')
  #csvResult = df.to_csv("d:\\gbtc.csv")
  mydf.to_csv('d:/' + ticker + '.csv')
  #csvresult = mydf.to_csv('gbtc.csv')
  
  return mydf

In [39]:
 #save_to_csv_from_yahoo("gbtc", "2021-01-01", "2021-11-30")

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-01-04,34.090000,36.599998,32.900002,35.080002,35.080002,22817800
2021-01-05,34.490002,39.540001,34.490002,39.410000,39.410000,19229000
2021-01-06,41.419998,44.000000,39.750000,43.930000,43.930000,40502900
2021-01-07,47.700001,48.410000,40.810001,44.970001,44.970001,32619300
2021-01-08,47.849998,48.650002,44.360001,44.419998,44.419998,23125700
...,...,...,...,...,...,...
2021-11-22,47.400002,47.400002,45.049999,45.150002,45.150002,4457200
2021-11-23,45.959999,47.000000,45.700001,46.880001,46.880001,3650200
2021-11-24,45.630001,47.709999,45.150002,46.419998,46.419998,3057300


# **Get data from CSV**

In [82]:
def get_df_from_csv(ticker):
    try:
        df = pd.read_csv('d:/' + ticker + '.csv')
    except FileNotFoundError:
        print("File not found")
    else:
        return df


# **Add daily return to DF**

In [47]:
# simple daily return: (Ending Prid - Beginning Price)/Beginning Price or (EP/BP)-1

def add_daily_return_to_df(df,ticker):
    df['Daily_Return'] = (df['Adj Close'] / df['Adj Close'].shift(1)) - 1
    df.to_csv('d:/' + ticker + '.csv')
    return df


# **Returns Total Return over Time**

In [71]:
def get_return_defined_time(df, syear, smonth, sday, eyear, emonth, eday):
    # Create string representations for the dates
    start = f"{syear}-{smonth}-{sday}"
    end = f"{eyear}-{emonth}-{eday}"
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Use a mask to grab data between defined dates
    mask = (df['Date'] >= start) & (df['Date'] <= end)
    
    # Get the mean of the column named daily return
    daily_ret = df.loc[mask]['Daily_Return'].mean()
    
    # Get the number of days between 2 dates
    df2 = df.loc[mask]
    days = df2.shape[0]

    # Return the total return between 2 dates
    return (days * daily_ret)

# **Matplotlib Finance**

In [95]:
# Creates a simple price / date plot between dates
def price_plot(ticker, syear, smonth, sday, eyear, emonth, eday):
    # Create string representations for the dates
    start = f"{syear}-{smonth}-{sday}"
    end = f"{eyear}-{emonth}-{eday}"
    
    try:

        df = pd.read_csv('D:/' + ticker + '.csv')
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        
        # Set data.index as DatetimeIndex
        df.index = pd.DatetimeIndex(df['Date'])
        
        # Define to only use data between provided dates
        df_sub = df.loc[start:end]
        
        # Convert to Numpy array
        df_np = df_sub.to_numpy()
        
        # Get adjusted close data from the 5th column
        np_adj_close = df_np[:,5]
        
        # Get date from the 1st
        date_arr = df_np[:,1]
        
        # Defines area taken up by the plot
        fig = plt.figure(figsize=(12,8),dpi=100)
        axes = fig.add_axes([0,0,1,1])
        
        # Define the plot line color as navy
        axes.plot(date_arr, np_adj_close, color='navy')
        
        # Set max ticks on the x axis
        axes.xaxis.set_major_locator(plt.MaxNLocator(8))
        
        # Add a grid, color, dashes(5pts 1 pt dashes separated by 2pt space)
        axes.grid(True, color='0.6', dashes=(5, 2, 1, 2))
        
        # Set grid background color
        axes.set_facecolor('#FAEBD7')


# **Test Functions**

In [96]:
GBTC = get_df_from_csv("GBTC")

#add_daily_return_to_df(GBTC,"GBTC")

#print(GBTC)

tot_ret = get_return_defined_time(GBTC, 2021, 1, 1, 2021, 11, 1)
print("Total Return :", tot_ret)

price_plot(GBTC, 2021, 1, 1, 2021, 12, 1)

Total Return : 0.758227863


UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U32')) -> dtype('<U32')